<a href="https://colab.research.google.com/github/nimamt/machine_learning/blob/master/pytorch/PyTorchBuildingTheNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Everything is in namespace torch.nn   
The NN is a "nn.Module" comprising other "nn.Module"s

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


initialize the neural network layers in __init__  
Every nn.Module subclass implements the operations on input data in the forward method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of NeuralNetwork, and move it to the device, and print its structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. . We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Logits: {logits}")
print(f"Pred_probab: {pred_probab}")

print(f"Predicted class: {y_pred}")

Logits: tensor([[-0.0265,  0.0334,  0.0545,  0.0365, -0.0606, -0.1254, -0.1351, -0.0455,
          0.0274,  0.0093]], grad_fn=<AddmmBackward0>)
Pred_probab: tensor([[0.0995, 0.1056, 0.1079, 0.1059, 0.0961, 0.0901, 0.0892, 0.0976, 0.1050,
         0.1031]], grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([2])


In [9]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [10]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1100, -0.1732,  0.7788, -0.1063,  0.0276, -0.2213,  0.2206, -0.0509,
         -0.2799, -0.3075,  0.1831, -0.4474, -0.1461,  0.3429, -0.3700,  0.3695,
          0.3400, -0.1779,  0.2926,  0.3530],
        [-0.1210, -0.1663,  0.5153, -0.4236,  0.2766, -0.2771,  0.0650,  0.1930,
         -0.0800, -0.0709,  0.5352, -0.5092, -0.1253,  0.3392, -0.8086,  0.1576,
          0.0481, -0.2922,  0.0989,  0.2460],
        [ 0.0541, -0.3886,  0.6981, -0.0341,  0.0348, -0.3995,  0.0525, -0.2674,
         -0.2706, -0.2987,  0.1123, -0.4461, -0.2327,  0.2846, -0.8422, -0.2055,
          0.4597, -0.2094,  0.1325,  0.1265]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.7788, 0.0000, 0.0276, 0.0000, 0.2206, 0.0000, 0.0000,
         0.0000, 0.1831, 0.0000, 0.0000, 0.3429, 0.0000, 0.3695, 0.3400, 0.0000,
         0.2926, 0.3530],
        [0.0000, 0.0000, 0.5153, 0.0000, 0.2766, 0.0000, 0.0650, 0.1930, 0.0000,
         0.0000, 0.5352, 0.0000, 0.0000, 0.3392, 0.00

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

--------- dim = 1

|  
|  
|  
|  
|  
dim = 0

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [15]:
pred_probab

tensor([[0.1034, 0.1184, 0.1028, 0.1198, 0.1103, 0.1057, 0.0669, 0.0903, 0.0820,
         0.1004],
        [0.1036, 0.1006, 0.0863, 0.0984, 0.1212, 0.1004, 0.0760, 0.1007, 0.0956,
         0.1172],
        [0.1015, 0.1042, 0.1016, 0.1095, 0.1012, 0.1007, 0.0819, 0.0991, 0.0984,
         0.1019]], grad_fn=<SoftmaxBackward0>)

In [16]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0088, -0.0004, -0.0163,  ...,  0.0263, -0.0165, -0.0068],
        [-0.0196, -0.0255,  0.0156,  ...,  0.0010, -0.0072,  0.0049]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0248, 0.0140], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0026, -0.0026, -0.0149,  ...,  0.0081,  0.0069,  0.0223],
        [ 0.0089, -0.0233,  0.0340,  ...,  0.0006, -0.0371,  0.0355]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si